In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold

Создание векторов из smiles

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")
model1 = AutoModel.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")
test_df_smiles = pd.read_csv('/content/smiles_test_set_public.csv')
df_smiles = pd.read_csv('/content/smiles_train_set.csv')
def smiles_to_vector(smiles_list):
    inputs = tokenizer(smiles_list, padding=True, truncation=True, return_tensors="pt", max_length=512)

    with torch.no_grad():
        outputs = model1(**inputs)

    vectors = outputs.last_hidden_state.mean(dim=1)
    return vectors


vector_list = []
test_vector_list = []
for smiles in test_df_smiles['smiles']:
    test_vector= smiles_to_vector([smiles])
    test_vector_list.append(test_vector.squeeze().numpy())
for smiles in df_smiles['smiles']:
    vector = smiles_to_vector([smiles])
    vector_list.append(vector.squeeze().numpy())

vector_df = pd.DataFrame(vector_list)
test_vector_df = pd.DataFrame(test_vector_list)


In [ ]:
smiles_vector=pd.concat([df_smiles, vector_df], axis=1)
columns = smiles_vector.columns.tolist()
col = [x for x in columns  if x not in ['smiles',	'oil_property_param_value', 'blend_id']]
new_df_smiles = smiles_vector.groupby('blend_id')[col].sum().reset_index()

test_smiles_vector=pd.concat([test_df_smiles, test_vector_df], axis=1)
test_columns = test_smiles_vector.columns.tolist()
test_col = [x for x in test_columns  if x not in ['smiles',	'oil_property_param_value', 'blend_id']]
new_test_df_smiles = test_smiles_vector.groupby('blend_id')[test_col].sum().reset_index()

new_df_smiles.to_csv('train_vectors.csv', index=False)
new_test_df_smiles.to_csv('test_vectors.csv', index=False)

Добавление свойств компонентов и финальная предобработка



In [ ]:
smiles_train = pd.read_csv('./smiles_train_set.csv')
smiles_test = pd.read_csv('./smiles_test_set_public.csv')
train = pd.read_csv('./train_data_fix_concat.csv')
test = pd.read_csv('./test_data_fix_concat.csv')
df_train_c = pd.read_csv('./blends_train.csv').drop(['Unnamed: 0','ad7e6027-00b8-4c27-918c-d1561f949ad8'], axis=1)
df_test_c = pd.read_csv('./blends_test.csv')
train_vectors = pd.read_csv('./train_vectors.csv')
test_vectors = pd.read_csv('./test_vectors.csv')

In [ ]:
df_test_c['target'] = -1
df_c = pd.concat([df_train_c, df_test_c])

df_train = train[['blend_id', 'component_property_param_title', 'component_property_param_value']].copy(deep=True)
df_test = test[['blend_id', 'component_property_param_title', 'component_property_param_value']].copy(deep=True)
df = pd.concat([df_train, df_test])

info = df.pivot_table(
    index = 'blend_id',
    values=['component_property_param_value'],
    columns=['component_property_param_title'],
    aggfunc=['mean', 'count', 'median', 'sum', 'min', 'max']
)
info.columns = ['_'.join(map(str, x)) for x in info.columns]
info.reset_index(inplace=True)

info_train = df_train.pivot_table(
    index = 'blend_id',
    values=['component_property_param_value'],
    columns=['component_property_param_title'],
    aggfunc=['mean', 'count', 'median', 'sum', 'min', 'max']
)
info_train.columns = ['_'.join(map(str, x)) for x in info_train.columns]
info_train.reset_index(inplace=True)

info_test = df_test.pivot_table(
    index = 'blend_id',
    values=['component_property_param_value'],
    columns=['component_property_param_title'],
    aggfunc=['mean', 'count', 'median', 'sum', 'min', 'max']
)
info_test.columns = ['_'.join(map(str, x)) for x in info_test.columns]
info_test.reset_index(inplace=True)

df = df_c.merge(info[list(set(info_train.columns) & set(info_test.columns))], on='blend_id').dropna(subset='target')
df[['oil_type']] = df[['oil_type']].fillna('0')

In [ ]:
df_train = df[df.target!=-1]
df_test = df[df.target==-1].drop(['target'], axis=1)

In [ ]:
df_train=df_train.merge(train_vectors, how='inner', on='blend_id')
df_test=df_test.merge(test_vectors, how='inner', on='blend_id')

In [ ]:
!pip install mljar-supervised

Обучение

In [ ]:
'''from supervised import AutoML
automl = AutoML(mode="Compete", algorithms=['Xgboost', 'CatBoost', 'LightGBM'], eval_metric='mae',total_time_limit=3*3600)
automl.fit(df_train.drop(['target', 'blend_id'], axis=1), df_train.target)'''

Подгрузка весов


In [ ]:
automl = AutoML(results_path='./AutoML_best')

In [ ]:
predicts = automl.predict(df_test.drop('blend_id', axis=1))

In [ ]:
submission = pd.DataFrame(zip(df_test.blend_id, predicts), columns=['blend_id', 'target'])
submission.to_csv('./submission.csv', sep=',', index=False)
submission